In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.metrics import AUC

In [ ]:
train_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train')
test_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test')

In [ ]:
labels=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
labels

In [ ]:
sample=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
sample

In [ ]:
def load_dicom(path):
    dicom=pydicom.read_file(path)
    data=dicom.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [ ]:
Img_size = 100
path0='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1wCE/Image-1.dcm'
img1=load_dicom(path0)
img2=cv2.resize(img1,(Img_size,Img_size)) 
print(img1.shape)
print(img2.shape)

In [ ]:
train_dir='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'

for fld in ['FLAIR','T1w','T1wCE','T2w']:
    print('*'*20, fld)
    trainset=[]
    trainlabel=[]
    fig_path = []
    my_label = []
    for i in tqdm(range(len(labels))):
        idt=labels.iloc[i]['BraTS21ID']
        idt2=('00000'+str(idt))[-5:]
        ii=0
        path_to_img = []
        img_nonzero_pix = []
        path=os.path.join(train_dir,idt2,fld) 
        for im in os.listdir(path):
            ii += 1
            img0=load_dicom(os.path.join(path,im))
            if np.max(img0)!=0:
                path_to_img.append(os.path.join(path,im))
                img_nonzero_pix.append(sum(sum(img0!=0)))
        idx = np.argsort(np.array(img_nonzero_pix))
        for k in idx[-16:]:
            fig_path.append(path_to_img[k])#[67:])
            my_label.append(str(labels.iloc[i]['MGMT_value']))
#             plt.figure(ii)
#             ax0 = plt.subplot(1,2,1)
#             ax0.imshow(img0)
#             ax1 = plt.subplot(1,2,2)
#             ax1.imshow(image)
#             plt.pause(1e-10)
    if fld=='FLAIR':
        flair_train_df = pd.DataFrame({'image_path':fig_path , 'target':my_label});
        flair_train_df.to_csv('./Flair_df.csv')
        display(flair_train_df.head())
    elif fld=='T1w':
        T1w_train_df = pd.DataFrame({'image_path':fig_path , 'target':my_label});
        T1w_train_df.to_csv('./T1w_df.csv')
        display(T1w_train_df.head())
    elif fld=='T1wCE':
        T1wCE_train_df = pd.DataFrame({'image_path':fig_path , 'target':my_label});
        T1wCE_train_df.to_csv('./T1wCE_df.csv')
        display(T1wCE_train_df.head())
    else:
        T2w_train_df = pd.DataFrame({'image_path':fig_path , 'target':my_label});
        T2w_train_df.to_csv('./T2w_df.csv')
        display(T2w_train_df.head())
        

In [ ]:
# import numpy as np
# import pandas as pd
# import pydicom
# import cv2
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras_preprocessing.image.dataframe_iterator import DataFrameIterator

# # tested on tf 2.1 

# class DCMDataFrameIterator(DataFrameIterator):
#     def __init__(self, *arg, **kwargs):
#         self.white_list_formats = ('dcm')
#         super(DCMDataFrameIterator, self).__init__(*arg, **kwargs)
#         self.dataframe = kwargs['dataframe']
#         self.x = self.dataframe[kwargs['x_col']]
#         self.y = self.dataframe[kwargs['y_col']]
#         self.color_mode = kwargs['color_mode']
#         self.target_size = kwargs['target_size']

#     def _get_batches_of_transformed_samples(self, indices_array):
#         # get batch of images
#         batch_x = np.array([self.read_dcm_as_array(dcm_path, self.target_size, color_mode=self.color_mode)
#                             for dcm_path in self.x.iloc[indices_array]])

#         batch_y = np.array(self.y.iloc[indices_array].astype(np.uint8))  # astype because y was passed as str

#         # transform images
#         if self.image_data_generator is not None:
#             for i, (x, y) in enumerate(zip(batch_x, batch_y)):
#                 transform_params = self.image_data_generator.get_random_transform(x.shape)
#                 batch_x[i] = self.image_data_generator.apply_transform(x, transform_params)
#                 # you can change y here as well, eg: in semantic segmentation you want to transform masks as well 
#                 # using the same image_data_generator transformations.

#         return batch_x, batch_y

#     @staticmethod
#     def read_dcm_as_array(dcm_path, target_size=(256, 256), color_mode='rgb'):
#         image_array = pydicom.dcmread(dcm_path).pixel_array
#         image_array = cv2.resize(image_array, target_size, interpolation=cv2.INTER_NEAREST)  #this returns a 2d array
#         image_array = np.expand_dims(image_array, -1)
#         if color_mode == 'rgb':
#             image_array = cv2.cvtColor(image_array, cv2.COLOR_GRAY2RGB)
#         return image_array

      
# # read data
# # Assuming it has two cols:
# # image_path: path to each image with its extension
# # target: labels (here it is 0s and 1s) -> binary classification
# # df = pd.read_csv("yourDfPath.csv", dtype=str)

# # split for testing
# # train_df, test_df = train_test_split(df, test_size=0.2)

# # augmentation parameters
# # you can use preprocessing_function instead of rescale in all generators
# # if you are using a pretrained network
# train_augmentation_parameters = dict(
#     rescale=1.0/255.0,
#     rotation_range=10,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='constant',
#     cval=0,
#     brightness_range = [0.8, 1.2],
#     validation_split = 0.2
# )

# valid_augmentation_parameters = dict(
#     rescale=1.0/255.0,
#     validation_split = 0.2
# )

# # test_augmentation_parameters = dict(
# #     rescale=1.0/255.0
# # )

# # training parameters
# BATCH_SIZE = 32
# CLASS_MODE = 'binary'
# COLOR_MODE = 'grayscale'
# TARGET_SIZE = (Img_size, Img_size)
# EPOCHS = 100
# SEED = 1337

# train_consts = {
#     'seed': SEED,
#     'batch_size': BATCH_SIZE,
#     'class_mode': CLASS_MODE,
#     'color_mode': COLOR_MODE,
#     'target_size': TARGET_SIZE,  
#     'subset': 'training'
# }

# valid_consts = {
#     'seed': SEED,
#     'batch_size': BATCH_SIZE,
#     'class_mode': CLASS_MODE,
#     'color_mode': COLOR_MODE,
#     'target_size': TARGET_SIZE, 
#     'subset': 'validation'
# }

# # test_consts = {
# #     'batch_size': 1,  # should be 1 in testing
# #     'class_mode': CLASS_MODE,
# #     'color_mode': COLOR_MODE,
# #     'target_size': TARGET_SIZE,  # resize input images
# #     'shuffle': False
# # }

# # Using the training phase generators 
# train_augmenter = ImageDataGenerator(**train_augmentation_parameters)
# valid_augmenter = ImageDataGenerator(**valid_augmentation_parameters)


# train_generator = DCMDataFrameIterator(dataframe=flair_train_df,
#                              x_col='image_path',
#                              y_col='target',
#                              image_data_generator=train_augmenter,
#                              **train_consts)

# valid_generator = DCMDataFrameIterator(dataframe=flair_train_df,
#                              x_col='image_path',
#                              y_col='target',
#                              image_data_generator=valid_augmenter,
#                              **valid_consts)

# # # define model architecture like how you normally do
# # model = ... 

# # # training
# # history = model.fit_generator(
# # 	generator=train_generator,
# # 	steps_per_epoch=len(train_generator),
# # 	epochs=EPOCHS,
# # 	validation_data=valid_generator,
# # 	validation_steps=len(valid_generator)
# # )



# # # Using the testing generator to evaluate the model after training
# # test_augmenter = ImageDataGenerator(**test_augmentation_parameters)
# # test_generator = DCMDataFrameIterator(dataframe=test_df,
# #                              x_col='image_path',
# #                              y_col='target',
# #                              image_data_generator=test_augmenter,
# #                              **test_consts)
# # test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))

In [ ]:
# model = Sequential()
# model.add(Conv2D(512,(4,4),input_shape = (Img_size,Img_size,1),activation = 'relu'))
# model.add(Conv2D(256,(2,2),activation = 'relu'))
# model.add(Conv2D(128,(2,2),activation = 'relu'))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [ ]:
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy',AUC()])

In [ ]:
# weights_filepath = './model_best_weights.hdf5'
# callback = EarlyStopping(monitor='val_auc', patience=30)
# callback2 = ModelCheckpoint(filepath=weights_filepath,save_weights_only=True,monitor='val_auc',mode='max',save_best_only=True)

In [ ]:
# model.summary()

In [ ]:
# # training
# history = model.fit_generator(
#     generator=train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=EPOCHS,
#     validation_data=valid_generator,
#     validation_steps=len(valid_generator),
#     callbacks=[callback,callback2] , verbose=1
# )


In [ ]:
# get_acc = his.history['accuracy']
# value_acc = his.history['val_accuracy']
# get_loss = his.history['loss']
# validation_loss = his.history['val_loss']
# get_auc = his.history['auc']
# validation_auc = his.history['val_auc']

# plt.figure(1,figsize=(20,5))
# epochs = range(len(get_acc))
# ax0 = plt.subplot(1,3,1)
# ax0.plot(epochs, get_acc, 'r', label='Train Accuracy')
# ax0.plot(epochs, value_acc, 'b', label='Validation Accuracy')
# # ax0.title('Training vs validation accuracy')
# ax0.legend(loc=0)

# epochs = range(len(get_loss))
# ax1 = plt.subplot(1,3,2)
# ax1.plot(epochs, get_loss, 'r', label='Train Loss')
# ax1.plot(epochs, validation_loss, 'b', label='Validation Loss')
# # ax1.title('Training vs validation loss')
# ax1.legend(loc=0)


# epochs = range(len(get_auc))
# ax2 = plt.subplot(1,3,3)
# ax2.plot(epochs, get_auc, 'r', label='Training AUC')
# ax2.plot(epochs, validation_auc, 'b', label='Validation AUC')
# # ax2.title('Training vs validation AUC')
# ax2.legend(loc=0)
# # ax2.figure()
# # ax2.show()